# Castro et al. (2019) Data Processing and Analysis

## Setup

In [2]:
import os
import sys
import glob
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [3]:
import re
import pickle
from collections import Counter
from functools import reduce

import pandas as pd
from collections import defaultdict
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib as mpl

In [4]:
from cvtk.cvtk import TemporalFreqs, TiledTemporalFreqs
from cvtk.cov import stack_temporal_covariances
import cvtk.variant_files as vf
from cvtk.gintervals import GenomicIntervals
from cvtk.pca import FreqPCA
from cvtk.plots import rep_plot_pca, correction_diagnostic_plot
from cvtk.utils import integerize
from cvtk.utils import extract_empirical_nulls_diagonals, extract_temporal_cov_diagonals
from cvtk.cov import stack_replicate_covariances, stack_temporal_covs_by_group
from cvtk.variant_files import VCFFile

In [5]:
%matplotlib inline
#%config InlineBackend.figure_format = 'svg'
mpl.rcParams['figure.figsize'] = (8.0, 4.0)
mpl.rcParams['figure.dpi'] = 200

# Varianta Data Loading

### Load in VCF data

In [6]:
vcf = VCFFile('../data/castro_et_al_2019/beagle_genMap.all.impute.vcf.gz')

reading file '../data/castro_et_al_2019/beagle_genMap.all.impute.vcf.gz'...
file '../data/castro_et_al_2019/beagle_genMap.all.impute.vcf.gz' loaded.
total time to load VCF file: 9.414049561818441 mins.


Remove fixed sites — those that are not polymorphic in any samples / timepoints. These just needlessly shrink the covariance towards zero.

### Sample Data

The samples names to line/generation mapping was not in a simple text file, but I found the relevant information in the vcftools commands that were at the beginning of the file `Longshanks_F0F17.summary_stats.tar.gz`. From this I created `samples.txt`, which is read in and parsed below.

In [7]:
samples = pd.read_csv("../data/castro_et_al_2019/samples.txt", header=None, names = ('line', 'individual'))
sample_map = {k:v for k, v in zip(samples['individual'], samples['line'])}

subpop_indices = defaultdict(list)
for i, k in enumerate(vcf.samples):
    subpop_indices[sample_map[k.decode()]].append(i)

From this, we can map the `vcf.geno_mat` table to subpopulation counts. 

In [8]:
counts_mat = vcf.count_alleles_subpops(subpop_indices)

/home/vinceb/projects/cvtk/cvtk/variant_files.py:310: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  self.mat = np.stack(counts_mat.values())


In [9]:
vcf.subpops

dict_keys(['Ctrl_F17', 'LS1_F17', 'LS2_F17', 'Ctrl_F0', 'LS1_F0', 'LS2_F0'])

Now we count the number of diploids in each sample.

In [10]:
ndiploids = [Counter(sample_map.values())[k] for k in vcf.subpops]

In [11]:
def parse_samples(x):
    line, gen = x.split('_')
    return (line, gen[1:])

design = [parse_samples(x) for x in vcf.subpops]

In [12]:
freq_mat_all = vcf.calc_freqs()

In [13]:
print("number of loci: ", freq_mat_all.shape[1])

number of loci:  31944210


With the frequencies calculated, now we filter out all non-segregating sites.

In [14]:
vcf.remove_fixed()
freq_mat = vcf.calc_freqs()
print("number of loci: ", freq_mat.shape[1])
print("loci not segregating removed: ", freq_mat_all.shape[1] - freq_mat.shape[1])

number of loci:  8162172
loci not segregating removed:  23782038


In [15]:
gi = vcf.build_gintervals()

## Replicate Covariance Analysis

In [16]:
tile_width = 10e6
gi.infer_seqlens()
tiles = GenomicIntervals.from_tiles(gi.seqlens, width=tile_width)

In [17]:
d = TiledTemporalFreqs(tiles, freqs=freq_mat, depths=vcf.N, diploids=ndiploids, samples=design, gintervals=gi)

In [18]:
d.samples

[('Ctrl', '0'),
 ('Ctrl', '17'),
 ('LS1', '0'),
 ('LS1', '17'),
 ('LS2', '0'),
 ('LS2', '17')]

In [19]:
autosomes = list(set(gi.intervals.keys()) - set('chrX'))

In [20]:
covs_cis = d.bootstrap_cov(B=5000, keep_seqids=autosomes, average_replicates=False, progress_bar=True)

In [21]:
covs_cis

array([[[ 0.07672857, -0.01895358, -0.01598237],
        [-0.01895358,  0.10834686,  0.01310542],
        [-0.01598237,  0.01310542,  0.12264452]],

       [[ 0.1029417 , -0.00699107, -0.00235625],
        [-0.00699107,  0.1369137 ,  0.02631757],
        [-0.00235625,  0.02631757,  0.16493679]],

       [[ 0.11928844,  0.00668343,  0.01398982],
        [ 0.00668343,  0.15533047,  0.0394337 ],
        [ 0.01398982,  0.0394337 ,  0.1894039 ]]])

In [21]:
with open('../data/castro_et_al_2019/covs_bootstrap_10e6.npy', 'wb') as f:
    np.save(f, covs_cis)

### Bootstrap the Convergence Correlation

In [30]:
convergence_corr = d.bootstrap_convergence_corr(B=5000, progress_bar=True)

/home/vinceb/projects/cvtk/cvtk/G.py:94: RuntimeWarning: invalid value encountered in sqrt
  sdmat = np.sqrt(varmat[:, tr, tc, :, :].mean(axis=1))


In [32]:
convergence_corr**2

array([[[[[0.02142034, 0.01486902, 0.02142034, 0.00746661, 0.01486902,
           0.00746661]]]],



       [[[[0.00271228, 0.00031146, 0.00271228, 0.0388111 , 0.00031146,
           0.0388111 ]]]],



       [[[[0.00372939, 0.01238449, 0.00372939, 0.08327051, 0.01238449,
           0.08327051]]]]])

## Analysis Excluding Chromosomes 5 and 10

In [24]:
autosomes_sans_chr5_and_chr10 = [chr for chr in autosomes if chr not in ('chr5', 'chr10')]

In [25]:
covs_sans_chr5_and_chr1_cis = d.bootstrap_cov(B=5000, keep_seqids=autosomes_sans_chr5_and_chr10, 
                                              average_replicates=False, progress_bar=True)

In [26]:
covs_sans_chr5_and_chr1_cis

array([[[ 0.06984107, -0.02055328, -0.01095816],
        [-0.02055328,  0.10789442,  0.01367839],
        [-0.01095816,  0.01367839,  0.15392888]],

       [[ 0.1029417 , -0.00699107, -0.00235626],
        [-0.00699107,  0.13691366,  0.02631749],
        [-0.00235626,  0.02631749,  0.16493667]],

       [[ 0.11667579,  0.00875339,  0.02117211],
        [ 0.00875339,  0.15783592,  0.0399416 ],
        [ 0.02117211,  0.0399416 ,  0.19842605]]])

In [27]:
with open('../data/castro_et_al_2019/covs_sans_chr5_and_chr1_bootstrap_10e6.npy', 'wb') as f:
    np.save(f, covs_sans_chr5_and_chr1_cis)